# Development: Train 3-Category Classifier
Train a 3-category classifier using only the non-road training samples.

Workflow sidesteps some problems with TensorFlow by simplifying the training and shifting some components—multiple epochs, callback functionality, validation, etc—to manual coding.

Currently, calls for training in just two epochs, one fast and one slow. May change.

Date: 2019-09-03  
Author: Peter Kerins  

## Preparation

### Import all modules

In [ ]:
# typical, comprehensive imports
import warnings
warnings.filterwarnings('ignore')
#
import os, sys
import json
import itertools, collections
import pickle
from pprint import pprint

import numpy as np
import pandas as pd
import geojson
import fiona
import ogr, gdal
# get_ipython().magic(u'matplotlib inline')
# import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model
# import math
# from tensorflow.keras import models
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.utils import to_categorical

# import tensorflow.keras as keras
# import tensorflow.keras.backend as K
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
# from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras.layers import Input, Add, Lambda
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, History
import h5py

import descarteslabs as dl

ULU_REPO = os.environ["ULU_REPO"]
if ULU_REPO not in sys.path:
    sys.path.append(ULU_REPO+'/utils')
    sys.path.append(ULU_REPO)
print(sys.path)

import util_descartes
#import util_ml
import util_rasters
import util_vectors
import util_workflow
import util_chips
import util_training
import util_network
import util_scoring
from catalog_generator import CatalogGenerator

### Set all user-defined variables

#### Base variables

In [ ]:
data_root='/data/phase_iv/'

resolution=5
# tile_resolution = resolution
# tile_size = 256
# tile_pad = 32

In [ ]:
subcatalog_name = 'india_all-data'

path_train = data_root+'models/'+subcatalog_name+'_train.csv'
path_valid = data_root+'models/'+subcatalog_name+'_valid.csv'

In [ ]:
shutdown_system = True

#### Chips variables
Only needed if selecting samples from master catalog, rather than loading subcatalog from file

In [ ]:
build_new = False

In [ ]:
processing_level = None
source = 's2'
#image_suffix = 'E'

s2_bands=['blue','green','red','nir','swir1','swir2','alpha']; s2_suffix='BGRNS1S2A'  # S2, Lx
# s1_bands=['vv','vh']; s1_suffix='VVVH'  

resampling='bilinear'
processing = None

label_suffix = 'aue'
label_lot = '0'

In [ ]:
place_images = {}
place_images['hindupur']=['U', 'V', 'W', 'X', 'Y', 'Z']
place_images['singrauli']=['O','P','Q','R','S','T','U']
place_images['vijayawada']=['H','I']
place_images['jaipur']=['T','U','W','X','Y','Z']
place_images['hyderabad']=['P','Q','R','S','T','U']
place_images['sitapur']=['Q','R','T','U','V']
place_images['kanpur']=['AH', 'AK', 'AL', 'AM', 'AN']
place_images['belgaum']=['P','Q','R','S','T']
place_images['parbhani']=['T','V','W','X','Y','Z']
place_images['pune']=['P', 'Q', 'T', 'U', 'S']
place_images['ahmedabad']= ['Z', 'V', 'W', 'X', 'Y', 'AA']
place_images['malegaon']=  ['V', 'W', 'X', 'Y', 'Z']
place_images['kolkata'] =  ['M','N','O','P','Q','R']
place_images['mumbai']=['P','Q','R','S','U','V']

In [ ]:
# category_label = {0:'Open Space',1:'Non-Residential',\
#                    2:'Residential Atomistic',3:'Residential Informal Subdivision',\
#                    4:'Residential Formal Subdivision',5:'Residential Housing Project',\
#                    6:'Roads',7:'Study Area',8:'Labeled Study Area',254:'No Data',255:'No Label'}

# cats_map = {}
# cats_map[0] = 0
# cats_map[1] = 1
# cats_map[2] = 2
# cats_map[3] = 2
# cats_map[4] = 2
# cats_map[5] = 3

#### Sample construction variables

In [ ]:
window = 17

In [ ]:


# bands stuff outdated! needs to be reconciled with catalog filtering
# will ignore for the moment since this is a bigger fix...
# haven't done any examples yet incorporating additional chips beyond s2
# into construction of a training sample
bands_vir=s2_bands[:-1]
bands_sar=None
bands_ndvi=None
bands_ndbi=None
bands_osm=None

# this can get updated when cloudmasking is added
# haze_removal = False


In [ ]:
# needs to be updated completely; bands stuff doesn't make sense right now
stack_label, feature_count = util_workflow.build_stack_label(
        bands_vir=bands_vir,
        bands_sar=bands_sar,
        bands_ndvi=bands_ndvi,
        bands_ndbi=bands_ndbi,
        bands_osm=bands_osm,)
print(stack_label, feature_count)

#### Model & training variables

In [ ]:
model_id = '3cat_all_new-workflow'
notes = 'using all data and cleaned up notebook'

In [ ]:
batch_size = 128
balancing = None

epochs_fast = 1
epochs_slow = 1

max_queue_size = 32
workers = 32

### Specify training & validation samples
Construct subcatalogs containing all target training & validation samples, __or__ load them from file, according to variable `build_new`

#### Option A: Construct subcatalogs by filtering master catalog

In [ ]:
if build_new:
    df = util_chips.load_catalog()
    print(len(df.index))


    mask = pd.Series(data=np.zeros(len(df.index),dtype='uint8'), index=range(len(df)), dtype='uint8')

    for place,image_list in place_images.items():
        for image in image_list:
            mask |= (df['city']==place) & (df['image']==image)

    # straight away remove road samples
    mask &= (df['lulc']!=6)

    # filter others according to specifications
    mask &= (df['gt_type']==label_suffix)
    mask &= (df['gt_lot']==int(label_lot))
    mask &= (df['source']==source)
    mask &= (df['resolution']==int(resolution))
    mask &= (df['resampling']==resampling)
    mask &= (df['processing']==str(processing).lower())

    print(np.sum(mask))

    df = df[mask]
    df.reset_index(drop=True,inplace=True)
    len(df)



    place_locales_paths = [
        '/data/phase_iv/models/3cat_Ahm_V-AA_place_locales.pkl',
        '/data/phase_iv/models/3cat_Bel_P-T_place_locales.pkl'       ,
        '/data/phase_iv/models/3cat_Hin_U-Z_place_locales.pkl'       ,
        '/data/phase_iv/models/3cat_Hyd_P-U_place_locales.pkl'       ,
        '/data/phase_iv/models/3cat_Jai_T-U+W-Z_place_locales.pkl'   ,
        '/data/phase_iv/models/3cat_Kan_AH+AK-AN_place_locales.pkl'  ,
        '/data/phase_iv/models/3cat_Mal_V-Z_place_locales.pkl'       ,
        '/data/phase_iv/models/3cat_Par_T+V-Z_place_locales.pkl',
        '/data/phase_iv/models/3cat_Pun_P-Q+S-U_place_locales.pkl',
        '/data/phase_iv/models/3cat_Sin_O-U_place_locales.pkl',
        '/data/phase_iv/models/3cat_Sit_Q-R+T-V_place_locales.pkl',
        '/data/phase_iv/models/3cat_Vij_H-I_place_locales.pkl',
        '/data/phase_iv/models/3cat_Kol_M-R_place_locales.pkl',
        '/data/phase_iv/models/3cat_Mum_P-V_place_locales.pkl'
    ]

    combined_place_locales = {}
    for place_locales_filename in place_locales_paths:
        with open(place_locales_filename, "rb") as f:
            place_locales = pickle.load(f,encoding='latin1')
        combined_place_locales.update(place_locales)
#     print(combined_place_locales)


    df_t, df_v = util_chips.mask_locales(df, combined_place_locales)
    print(len(df_t), len(df_v))

    # save the datasets for future use
    %time df_t.to_csv(path_train,index=False)
    %time df_v.to_csv(path_valid,index=False)

#### Option B: Load existing subcatalog

In [ ]:
if not build_new:
    df_t = pd.read_csv(path_train, encoding='utf8')
    df_v = pd.read_csv(path_valid, encoding='utf8')
    print(len(df_t), len(df_v))

### Inspect selected samples

In [ ]:
print('train:')
print(util_training.calc_category_counts(df_t,remapping=None), len(df_t))
print('valid:')
print(util_training.calc_category_counts(df_v,remapping=None), len(df_v))
print()
if build_new:
    print('all:')
    print(util_training.calc_category_counts(df,remapping=None), len(df))

---

## Model

### Build loss function

#### Generate class weighting information

In [ ]:
# df2=pd.concat([df_t,df_v])

In [ ]:
category_weights = util_training.generate_category_weights(df_t,remapping='standard',log=False,mu=1.0,max_score=None)
print(category_weights.items())
weights = list(zip(*category_weights.items()))[1]
print(weights)

In [ ]:
category_weights_filename = data_root+'models/'+model_id+'_category_weights.pkl'

if os.path.exists(category_weights_filename):
    raise Exception('Cannot save category weights: file already exists at specified path ('+category_weights_filename+')')
else:
    pickle.dump(category_weights, open(category_weights_filename, 'wb'))

#### Use weights to create weighted categorical crossentropy loss function

In [ ]:
loss = util_training.make_loss_function_wcc(weights)

### Build convolutional neural network and prepare it for training

In [ ]:
#hardcoded params
network=util_network.build_xmodel(input_shape=(17,17,6),output_nodes=3,input_conv_block=True)
util_network.compile_network(network, loss, LR=0.001)

---

## Training

### Conduct "fast" training with high learning rate

#### Create sample "generators" (Keras _sequence_ objects) to serve samples

In [ ]:
generator_t = CatalogGenerator(df_t,remapping='3cat',look_window=window,batch_size=batch_size,one_hot=3)
generator_v = CatalogGenerator(df_v,remapping='3cat',look_window=window,batch_size=batch_size,one_hot=3)

#### Initial training

In [ ]:
# train fast
#history_fast = network.fit(X_train, Y_t_cat, batch_size=batch_size, epochs=epochs, validation_data=(X_valid, Y_v_cat), shuffle=True,callbacks=callbacks)
#docs: fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None,
                    #class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
history_fast = network.fit_generator(generator_t, epochs=epochs_fast, callbacks=None, steps_per_epoch=generator_t.steps,
                                    #validation_data=generator_v, validation_steps=generator_v.steps,
                                    shuffle=True,use_multiprocessing=True,max_queue_size=max_queue_size,workers=workers,)

# plt.plot(history_fast.history['val_acc'])
# plt.show()
# plt.plot(history_fast.history['val_loss'])
# plt.show()

#### Store trained weights

In [ ]:
fast_weights_path = data_root + 'models/' + model_id + '_weights_fast' + '.hd5'
print(fast_weights_path)
network.save_weights(fast_weights_path)

---

### Rebuild model and conduct "slow" training with lower learning rate

In [ ]:
#hardcoded params
network=util_network.build_xmodel(input_shape=(17,17,6),output_nodes=3,input_conv_block=True)
# load weights from fast learning
# network.load_weights(fast_weights_path)

# util_network.compile_network(network, loss, LR=0.0001)

#### Load trained weights and prepare network for additional training

In [ ]:
network.load_weights(fast_weights_path)
util_network.compile_network(network, loss, LR=0.0001)

#### Reset generators

In [ ]:
generator_t.reset()
generator_v.reset()

#### Additional training

In [ ]:
history_slow = network.fit_generator(generator_t, epochs=epochs_slow, callbacks=None, steps_per_epoch=generator_t.steps,
                                    #validation_data=generator_v, validation_steps=generator_v.steps,
                                    shuffle=True,use_multiprocessing=True,max_queue_size=max_queue_size,workers=workers,)

# plt.plot(history_slow.history['val_acc'])
# plt.show()
# plt.plot(history_slow.history['val_loss'])
# plt.show()

#### Store further trained weights

In [ ]:
slow_weights_path = data_root + 'models/' + model_id + '_weights_slow' + '.hd5'
print(slow_weights_path)
network.save_weights(slow_weights_path)

#### Store entire network object

In [ ]:
network_filename = data_root+'models/'+model_id+'.hd5'

if os.path.exists(network_filename):
    raise Exception('Cannot save network: file already exists at specified path ('+network_filename+')')
else:
    network.save(network_filename)

---

## Scoring

### Apply model to training and validation data

In [ ]:
generator_t.reset()
#predict_generator(generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
predictions_t = network.predict_generator(generator_t, steps=generator_t.steps, verbose=1,
                  use_multiprocessing=True,max_queue_size=max_queue_size,workers=workers,)
print(predictions_t.shape)

generator_v.reset()
#predict_generator(generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
predictions_v = network.predict_generator(generator_v, steps=generator_v.steps, verbose=1,
                  use_multiprocessing=True,max_queue_size=max_queue_size,workers=workers,)
print(predictions_v.shape)

In [ ]:
Yhat_t = predictions_t.argmax(axis=-1)
print(Yhat_t.shape)
Yhat_v = predictions_v.argmax(axis=-1)
print(Yhat_v.shape)

### Extract corresponding _actual_ ground-truth values directly from catalog

In [ ]:
Y_t = generator_t.get_label_series().values
print(Y_t.shape)
Y_v = generator_v.get_label_series().values
print(Y_v.shape)

### Generate typical scoring information

In [ ]:
print("evaluate training")
# hardcoded categories
categories=[0,1,2]
train_confusion = util_scoring.calc_confusion(Yhat_t,Y_t,categories)
train_recalls, train_precisions, train_accuracy = util_scoring.calc_confusion_details(train_confusion)

# Calculate f-score
beta = 2
train_f_score = (beta**2 + 1) * train_precisions * train_recalls / ( (beta**2 * train_precisions) + train_recalls )
train_f_score_open = train_f_score[0] 
train_f_score_nonres = train_f_score[1]  
train_f_score_res = train_f_score[2]  
train_f_score_roads = None#train_f_score[3]  
train_f_score_average = np.mean(train_f_score)

In [ ]:
print ("evaluate validation")
valid_confusion = util_scoring.calc_confusion(Yhat_v,Y_v,categories)
valid_recalls, valid_precisions, valid_accuracy = util_scoring.calc_confusion_details(valid_confusion)

# Calculate f-score
valid_f_score = (beta**2 + 1) * valid_precisions * valid_recalls / ( (beta**2 * valid_precisions) + valid_recalls )
valid_f_score_open = valid_f_score[0] 
valid_f_score_nonres = valid_f_score[1] 
valid_f_score_res = valid_f_score[2] 
valid_f_score_roads = None# valid_f_score[3] 
valid_f_score_average = np.mean(valid_f_score)

In [ ]:
# expanding lists to match expected model_record stuff
train_recalls_expanded = [train_recalls[0],train_recalls[1],train_recalls[2],None]
valid_recalls_expanded = [valid_recalls[0],valid_recalls[1],valid_recalls[2],None]
train_precisions_expanded = [train_precisions[0],train_precisions[1],train_precisions[2],None]
valid_precisions_expanded = [valid_precisions[0],valid_precisions[1],valid_precisions[2],None]

### Record experiment configuration and results

In [ ]:
util_scoring.record_model_creation(
    model_id, notes, place_images, label_suffix+label_lot, resolution, stack_label, feature_count, 
    window, generator_t.remapping, balancing, 
    network.get_config(), epochs, batch_size,
    train_confusion, train_recalls_expanded, train_precisions_expanded, train_accuracy,
    train_f_score_open, train_f_score_nonres, train_f_score_res, train_f_score_roads, train_f_score_average,
    valid_confusion, valid_recalls_expanded, valid_precisions_expanded, valid_accuracy,
    valid_f_score_open, valid_f_score_nonres, valid_f_score_res, valid_f_score_roads, valid_f_score_average,)

---

## Cleanup

In [ ]:
if shutdown_system:
    print('\n'*4)
    print("========================")
    print("========================")
    print("==== sudo poweroff =====")
    print("========================")
    print("========================")
    print('\n'*4)
    print("!dev-goodbye!")

    os.system('sudo poweroff')